In [1]:
%load_ext lab_black

In [2]:
import pandas as pd

In [3]:
def normalize_group(cms):
    if cms in {"eSchoolView", "Linq Websites"}:
        return "Linq Websites (formerly eSchoolView)"
    return cms

In [4]:
RAW_DATA_FILEPATH = "../data/what_cms_data.csv"
INDY_DATA_FILEPATH = "../data/indy_data.csv"
PUBLIC_FILEPATH = "../data/public_data.csv"

In [5]:
INDY_CUTOFF = 6
PUBLIC_CUTOFF = 15

In [6]:
COLUMNS = ["WHOCMS", "WHOCMS Previous 2021", "WHOCMS Previous 2020"]

In [7]:
EXCLUDE_INDUSTRIES = {"Higher Education", "International School"}

In [8]:
CMS = {
    "Finalsite",
    "SchoolPointe",
    "SOCS",
    "Blackboard",
    "SchoolMessenger",
    "Edlio",
    "Apptegy",
    "Gabbart or North American Division",
    "Educational Networks",
    "eSchoolView",
    "School Blocks",
    "Campus Suite",
    "School Loop",
    "Catapult CMS",
    "Blackbaud OnMessage",
    "Blackbaud",
    "School Webmasters",
    "eChalk",
    "SCHOOLinSITES",
    "Sitefinity",
    "eCatholic",
    "CivicEngage",
    "Linq Websites",
    "Revize",
    "elcomCMS",
    "Percussion",
    "OU Campus",
    "Moodle",
    "Aeries",
    "CMS4Schools",
    "Cnyric",
    "Foxbright",
    "Joomla",
    "Drupal",
}

In [9]:
df = (
    pd.read_csv(RAW_DATA_FILEPATH, encoding="ISO-8859-1")
    .loc[lambda df: ~df["Industry"].isin(EXCLUDE_INDUSTRIES), COLUMNS + ["Industry"],]
    .assign(
        cms_2020_07=lambda df: df["WHOCMS Previous 2020"].apply(normalize_group),
        cms_2020_11=lambda df: df["WHOCMS Previous 2021"].apply(normalize_group),
        cms_2021_08=lambda df: df["WHOCMS"].apply(normalize_group),
    )
    .drop(COLUMNS, axis=1)
    .dropna()
)

In [10]:
public_df = (
    df[df["Industry"] == "Public School District"]
    .drop("Industry", axis=1)
    .apply(lambda s: s.value_counts())
    .fillna(0)
    .reset_index()
    .loc[lambda df: df["index"].isin(CMS)]
    .loc[lambda df: df["cms_2021_08"] > PUBLIC_CUTOFF]
    .rename(columns={"index": "cms"})
)

In [11]:
indy_df = (
    df[df["Industry"] == "Independent School"]
    .drop("Industry", axis=1)
    .apply(lambda s: s.value_counts())
    .fillna(0)
    .reset_index()
    .loc[lambda df: df["index"].isin(CMS)]
    .loc[lambda df: df["cms_2021_08"] > INDY_CUTOFF]
    .rename(columns={"index": "cms"})
)

In [12]:
public_df.to_csv(PUBLIC_FILEPATH, index=False)
indy_df.to_csv(INDY_DATA_FILEPATH, index=False)